for breif matrix explanations read-> https://github.com/USEPA/USEEIO_API/blob/master/doc/data_format.md

In [43]:
#BASE URL AND KEY
#Use the requests library
import csv
import zipfile, io
import os
import pathlib
import requests as r
import pandas as pd
pd.options.display.max_rows = 999

base_url ='https://smmtool.app.cloud.gov/api/' 
api_headers = {}
# api-key for USEEIO
with open(".USEEIO_API_KEY", 'r') as KEY_FILE:
    api_headers['x-api-key']=KEY_FILE.read()

In [17]:
# AVAILABLE MODELS
# Show the available models in the native JSON output
models = r.get(base_url+'/models',headers=api_headers)
models_json = models.json()

# MODEL NAME
#Use the first available model
model_name = models_json[1]['id']

#MODEL SECTORS
#Get the sectors
model_sectors = r.get(base_url+model_name+'/sectors',headers=api_headers)
model_sectors_json = model_sectors.json()
model_sector_ids = []
for i in model_sectors_json:
    model_sector_ids.append(i['id'])
first_sector_id = model_sector_ids[0]
first_sector_name = model_sectors_json[0]['name']
print('The '+ model_name +' model has ' + str(len(model_sectors_json)) + ' sectors.')

# INDICATORS
model_indictrs_response = r.get(base_url+model_name+'/indicators',headers=api_headers)
model_indictrs_json = model_indictrs_response.json()
HC_indictr_id = model_indictrs_json[8]['id']
model_indictrs_names = []
model_indictrs_ids = []
for i in model_indictrs_json:
    model_indictrs_names.append(i['name'])
    model_indictrs_ids.append(i['id'])
print('The '+ model_name +' model has ' + str(len(model_indictrs_json)) + ' indicators. The names are:')
print(model_indictrs_names)

The GAUSEEIO model has 776 sectors.
The GAUSEEIO model has 24 indicators. The names are:
['Acid Rain', 'Energy', 'Freshwater Aquatic Ecotoxicity', 'Eutrophication', 'Food Waste', 'Global Climate Change', 'Hazardous Air Pollutants', 'Hazardous Waste', 'Human Health Cancer', 'Human Health Noncancer', 'Human Health - Respiratory Effects', 'Human Health Cancer and Noncancer', 'Employment', 'Land', 'Metals', 'Minerals and Metals', 'Muncipal Solid Waste', 'Nonrenewable Energy', 'Ozone Depletion', 'Pesticides', 'Renewable Energy', 'Smog Formation', 'Value Added', 'Water']


In [25]:
# D0_response = r.get(base_url+model_name+'/matrix/D?row=0',headers=api_headers)
# D0 = D0_response.json()
# sorted(D0)

D0_response = r.get(base_url+model_name+'/matrix/D',headers=api_headers)
D0 = D0_response.json()

# sorted(D0[0])

In [51]:
#
# Collect ALL Indicators from ALL Industr Sectors and Limit to Georgia
#

# D0_response = r.get(base_url+model_name+'/matrix/D?row=0',headers=api_headers)
D0_response = r.get(base_url+model_name+'/matrix/D',headers=api_headers)
D0 = D0_response.json()

indicator_data = {}
# Add the industry sector IDs
# indicator_data.update({'industry':model_sector_ids})
indicator_data.update({'industry_code':[_.split('/')[0] for _ in model_sector_ids]})
indicator_data.update({'industry_detail':[_.split('/')[1] for _ in model_sector_ids]})
indicator_data.update({'industry_region':[_.split('/')[2] for _ in model_sector_ids]})


for key, values_list in zip(model_indictrs_ids, D0):
    indicator_data.update({key:values_list})

all_indic_all_indust = pd.DataFrame(data=indicator_data)
# all_indic_all_indust.loc[0:20,['industry_code', 'industry_detail', 'ACID']].sort_values(by='ACID', ascending=False)

all_indic_all_indust.head()

,industry_code,industry_detail,industry_region,ACID,ENRG,ETOX,EUTR,FOOD,GCC,HAPS,...,METL,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR
0,1111a0,"fresh soybeans, canola, flaxseeds, and other o...",us-ga,0.006991,0.000000,27.141609,0.008084,0.000021,0.549898,0.000099,...,5.481034e-09,0.0,0.000083,0.0,0.000005,0.000380,0.000000,0.048809,0.471064,0.009608
1,1111b0,"fresh wheat, corn, rice, and other grains",us-ga,0.023198,0.000000,9.711579,0.033350,0.000130,1.703382,0.000131,...,8.166104e-07,0.0,0.000506,0.0,0.000006,0.000570,0.000000,0.059359,-0.002548,0.461626
2,111200,"fresh vegetables, melons, and potatoes",us-ga,0.005637,0.002340,22.179052,0.005695,0.001000,0.354328,0.000349,...,2.390340e-06,0.0,0.003883,0.0,0.000012,0.000951,0.002340,0.017317,0.604597,0.451799
3,111300,fresh fruits and tree nuts,us-ga,0.003150,0.000000,13.995239,0.000306,0.001330,0.235407,0.000091,...,1.030074e-05,0.0,0.005168,0.0,0.000006,0.000209,0.000000,0.016048,0.605345,0.403344
4,111400,"greenhouse crops, mushrooms, nurseries, and fl...",us-ga,0.000780,0.000004,0.006251,0.000097,0.001812,0.657274,0.000096,...,1.858372e-08,0.0,0.007078,0.0,0.000005,0.000046,0.000004,0.032099,0.534020,0.056251


---
### Load USEEIO <--> NAICS 2012 Concordance

In [68]:
# Load USEEIO <--> NAICS Concordance
data_dir = pathlib.Path.cwd().parent / 'data_raw'
useeio_keys = pd.read_csv(data_dir / 'USEEIO-NAICS-Concordance_2012.csv', header=5).loc[0:958,:]


def expand_keys(key_string):
    
    keys = []
    
    for key_subset in key_string.split(','):

        key_subset = key_subset.strip()
        try:
            start = key_subset.split('-')[0]
            end = key_subset.split('-')[1]

            try:
                key = int(start)
                keys.append(key)

                while str(key)[-1] != end:
                    key += 1
                    keys.append(key)
                    
            except ValueError:
                keys.append('*')
                
        except IndexError:
            try:
                keys.append(int(key_subset))
            except ValueError:
                keys.append('*')
                
    return keys


useeio_naic_concordance = useeio_keys[['Detail', 'Related 2012 NAICS Codes']].dropna().reset_index(drop=True)
useeio_naic_concordance['Detail'] = useeio_naic_concordance['Detail'].apply(lambda x: str.lower(x))

useeio_naic_concordance = useeio_naic_concordance.rename(columns={'Detail': 'industry_code'})

useeio_naic_concordance['relavant_naics_codes'] = useeio_naic_concordance.apply(lambda x: expand_keys(x["Related 2012 NAICS Codes"]), axis=1)
useeio_naic_concordance.head()

,industry_code,Related 2012 NAICS Codes,relavant_naics_codes
0,1111a0,11111-2,"[11111, 11112]"
1,1111b0,"11113-6, 11119","[11113, 11114, 11115, 11116, 11119]"
2,111200,1112,[1112]
3,111300,1113,[1113]
4,111400,1114,[1114]


In [70]:
all_indic_all_indust = all_indic_all_indust[
    all_indic_all_indust['industry_code'].isin(useeio_naic_concordance['industry_code'].tolist())
]

In [71]:
all_indic_all_indust.head()

,industry_code,industry_detail,industry_region,ACID,ENRG,ETOX,EUTR,FOOD,GCC,HAPS,...,METL,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR
0,1111a0,"fresh soybeans, canola, flaxseeds, and other o...",us-ga,0.006991,0.000000,27.141609,0.008084,0.000021,0.549898,0.000099,...,5.481034e-09,0.0,0.000083,0.0,0.000005,0.000380,0.000000,0.048809,0.471064,0.009608
1,1111b0,"fresh wheat, corn, rice, and other grains",us-ga,0.023198,0.000000,9.711579,0.033350,0.000130,1.703382,0.000131,...,8.166104e-07,0.0,0.000506,0.0,0.000006,0.000570,0.000000,0.059359,-0.002548,0.461626
2,111200,"fresh vegetables, melons, and potatoes",us-ga,0.005637,0.002340,22.179052,0.005695,0.001000,0.354328,0.000349,...,2.390340e-06,0.0,0.003883,0.0,0.000012,0.000951,0.002340,0.017317,0.604597,0.451799
3,111300,fresh fruits and tree nuts,us-ga,0.003150,0.000000,13.995239,0.000306,0.001330,0.235407,0.000091,...,1.030074e-05,0.0,0.005168,0.0,0.000006,0.000209,0.000000,0.016048,0.605345,0.403344
4,111400,"greenhouse crops, mushrooms, nurseries, and fl...",us-ga,0.000780,0.000004,0.006251,0.000097,0.001812,0.657274,0.000096,...,1.858372e-08,0.0,0.007078,0.0,0.000005,0.000046,0.000004,0.032099,0.534020,0.056251


In [73]:
all_indic_all_indust = (all_indic_all_indust
 .set_index('industry_code')
 .join(useeio_naic_concordance
      .set_index('industry_code')
      )
)